# tool의 병렬적 사용

- 다중 도구를 사용하는 가이드에서 우리는 여러 도구 사이에서 선택하는 함수 호출 체인을 구축하는 방법을 보았습니다. 
- 2023년 가을에 출시된 OpenAI 모델과 같은 일부 모델들은 병렬 함수 호출을 지원하는데, 이는 단일 모델 호출에서 여러 함수(또는 동일한 함수를 여러 번)를 호출할 수 있게 해줍니다. 
- 이전에 다중 도구 가이드에서 사용된 체인은 실제로 이미 이 기능을 지원합니다.


In [2]:
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

In [3]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [7]:
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    # RunnableLambda,
    # RunnableMap,
    # RunnablePassthrough,
)

tools = [multiply, exponentiate, add]
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

In [8]:
chain.invoke(
    "What's 23 times 7, and what's five times 18 and add a million plus a billion and cube thirty-seven"
)

[{'name': 'multiply',
  'args': {'first_int': 23, 'second_int': 7},
  'id': 'call_E3AknrxKDWgSVpvUJgwMSrNH',
  'output': 161},
 {'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 18},
  'id': 'call_oskMq1GEh3UTyRxhOKAfvtYn',
  'output': 90},
 {'name': 'add',
  'args': {'first_int': 1000000, 'second_int': 1000000000},
  'id': 'call_FVPlQwIqjDRSM2GwOhCpgofq',
  'output': 1001000000},
 {'name': 'exponentiate',
  'args': {'base': 37, 'exponent': 3},
  'id': 'call_EYkTecqeujQygkvF1J7eu3Th',
  'output': 50653}]